<h2>SISTEMA DE PREDICCIÓN DE MORTALIDAD DE PACIENTES<br>
CON COVID-19</h2>
<h3>Integrantes:<br>
Ronald Daniel Prieto Sarmiento<br>
Fredy Enmanuel Vila Solano<br>
Deiner Alfonso

In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<h2>Carga de los datos crudos</h2>

In [169]:
##Carga de los datos en crudo
datos = pd.read_csv('Casos_positivos_de_Covid-19_en_el_departamento_de_Antioquia.csv' , sep=",")

C:\Users\fredy\AppData\Local\Temp\ipykernel_19160\4185255095.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv('Casos_positivos_de_Covid-19_en_el_departamento_de_Antioquia.csv' , sep=",")


<h4>Muestra de los datos cargados</h4>

In [170]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951116 entries, 0 to 951115
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   fecha reporte web             951116 non-null  object 
 1   ID de caso                    951116 non-null  object 
 2   Fecha de notificación         951116 non-null  object 
 3   Código DIVIPOLA departamento  951116 non-null  int64  
 4   Nombre departamento           951116 non-null  object 
 5   Código DIVIPOLA municipio     951116 non-null  object 
 6   Nombre municipio              951116 non-null  object 
 7   Edad                          951116 non-null  int64  
 8   Unidad de medida de edad      951116 non-null  int64  
 9   Sexo                          951116 non-null  object 
 10  Tipo de contagio              951116 non-null  object 
 11  Ubicación del caso            946845 non-null  object 
 12  Estado                        946845 non-nul

<h3>Análisis exploratorio de los datos<h/3>

In [171]:
##La informacion que tiene el dataset, esto es para el punto 2
##estos son las variables que vamos a tener en cuenta para hacer el entrenamiento, pero tenemos que filtrar cuales nos sirven
datos.keys()

Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')

In [172]:
##Tambien hay forma de mirar que parametros no tienen informacion para eliminarlos y no tenerlos en cuenta:
datos.isna().sum()

fecha reporte web                    0
ID de caso                           0
Fecha de notificación                0
Código DIVIPOLA departamento         0
Nombre departamento                  0
Código DIVIPOLA municipio            0
Nombre municipio                     0
Edad                                 0
Unidad de medida de edad             0
Sexo                                 0
Tipo de contagio                     0
Ubicación del caso                4271
Estado                            4271
Código ISO del país             950266
Nombre del país                 950266
Recuperado                        3774
Fecha de inicio de síntomas       1486
Fecha de muerte                 928186
Fecha de diagnóstico               334
Fecha de recuperación            22494
Tipo de recuperación             22503
Pertenencia étnica                  30
Nombre del grupo étnico         948245
dtype: int64

<p>Debido al gran numero de elementos nulos o NaN, estos elementos son los primeros eliminador para la consideración del entrenamiento del modelo.<br>
- Código ISO del país<br>
- Nombre del país<br>
- Fecha de muerte<br>
- Nombre del grupo étnico

In [124]:
# este parametro tambien es importante, ya que nos permite identificar el estado del paciente y realizar el entrenamiento del algoritmo:
datos['Estado'].value_counts()

Leve         924822
Fallecido     18659
leve           3326
Moderado         32
Grave             6
Name: Estado, dtype: int64

<h3>Análisis y selección de información de información de interes

<p>Acontinuacion se mostraran cuales son los parametros que <b>NO</b> nos sirven en el dataset, para asi eliminarlos y filtrar la infarmacion mas importante:<br>
<ul>
<li> <b>Fecha reporte web</b>  : No da información relevante.</li>
<li> <b>ID de caso</b> : No es relevante para determinar la mortalidad de un paciente.</li>
<li> <b>Fecha de notificación</b> : No da información relevante.</li>
<li> <b>Nombre municipio</b> : Es linalmente dependiente con el campo Código DIVIPOLA municipio, y este no fue eliminado, dado que un paciente es más probable que muera si hace parte de un municipio, donde hubo alta tasa de contangios.</li>
<li> <b>Ubicación del caso </b> : Se elimina ya que da información de la etiqueta que se usará para la clasifiación de los datos.</li>
<li> <b>Nombre departamento</b> : Todos son del departamento de Antioquia, por tanto no es información que ayude a discrimar.</li>
<li> <b>Nombre del país</b> : Es linalmente dependiente con el campo Código ISO del país.</li>
<li> <b>Nombre del grupo étnico</b> : Se elimina dado que es mucho mayor la cantidad de información no entregada de este parámetro.</li>
<li> <b>Tipo de recuperación</b> : Este campo es linelamente independiente, con Recuperado, sin embargo este parámetro tiene más información sin clasificar que puede corresponder a fallecidos o activos (NAN=21157), por tanto se elimina este parámetro.</li>
<li> <b>Fecha de diagnóstico</b> : No da información relevante, dado que no importa cuándo se diagnostique sino la fecha de inicio de sintomas, por tanto tienen relación.</li>
<li> <b>Fecha de recuperación</b> : Nos indica que el paciente ya se encuentra recuperado, entonces no se tiene en cuenta.</li>
<li> <b>Fecha de muerte</b> : Esta etiqueta informa que ya el paciente esta fallecido.</li>

<li> <b>Estado </b> : Contiene información relacionada con la etiqueta que se usará para la clasifiación de los datos, por lo que es necesario eliminarla.</li>

</ul>

In [173]:
datos["Edad en días"]=datos.Edad[(datos.iloc[:,8]==3)] 
datos["Edad en meses"]=datos.Edad[(datos.iloc[:,8]==2)]
datos["Edad en años"]=datos.Edad[(datos.iloc[:,8]==1)]
datos['Edad en años'].fillna(0 , inplace=True)
datos['Edad en meses'].fillna(0 , inplace=True)
datos['Edad en días'].fillna(0 , inplace=True)
datos.head()

,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,...,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico,Edad en días,Edad en meses,Edad en años
0,2021-01-12 00:00:00,"1,805,802",2021-01-08 00:00:00,5,ANTIOQUIA,"5,001",MEDELLIN,34,1,F,...,2021-01-06 00:00:00,NaN,2021-01-10 00:00:00,2021-01-20 00:00:00,PCR,6.0,NaN,0.0,0.0,34.0
1,2021-01-12 00:00:00,"1,805,803",2021-01-08 00:00:00,5,ANTIOQUIA,"5,001",MEDELLIN,20,1,F,...,2021-01-06 00:00:00,NaN,2021-01-10 00:00:00,2021-01-20 00:00:00,Tiempo,6.0,NaN,0.0,0.0,20.0
2,2021-01-12 00:00:00,"1,805,804",2021-01-09 00:00:00,5,ANTIOQUIA,"5,360",ITAGUI,46,1,F,...,2021-01-07 00:00:00,NaN,2021-01-09 00:00:00,2021-01-22 00:00:00,Tiempo,6.0,NaN,0.0,0.0,46.0
3,2021-01-12 00:00:00,"1,805,805",2021-01-08 00:00:00,5,ANTIOQUIA,"5,088",BELLO,40,1,M,...,2021-01-06 00:00:00,NaN,2021-01-08 00:00:00,2021-01-20 00:00:00,Tiempo,6.0,NaN,0.0,0.0,40.0
4,2021-01-12 00:00:00,"1,805,806",2021-01-08 00:00:00,5,ANTIOQUIA,"5,360",ITAGUI,38,1,M,...,2021-01-05 00:00:00,NaN,2021-01-09 00:00:00,2021-01-19 00:00:00,Tiempo,6.0,NaN,0.0,0.0,38.0


<h3>Imputación de datos o eliminación.

<p>Al marginar los datos que no usaremos (infomración que se decidó, no sirve para este ejercicio), la base de datos queda con nueve (9) parametros los cuales se deben organizar para poder crear el data set de entrenamiento para los modelos.</p>

In [167]:
datos.isna().sum()

fecha reporte web                    0
ID de caso                           0
Fecha de notificación                0
Código DIVIPOLA departamento         0
Nombre departamento                  0
Código DIVIPOLA municipio            0
Nombre municipio                     0
Edad                                 0
Unidad de medida de edad             0
Sexo                                 0
Tipo de contagio                     0
Ubicación del caso                4271
Estado                            4271
Código ISO del país             950266
Nombre del país                 950266
Recuperado                        3774
Fecha de inicio de síntomas       1486
Fecha de muerte                 928186
Fecha de diagnóstico               334
Fecha de recuperación            22494
Tipo de recuperación             22503
Pertenencia étnica                  30
Nombre del grupo étnico         948245
Edad en días                         0
Edad en meses                        0
Edad en años             

<p>Los principales factores a descartar son los que posean mayor numero de datos nulos, ya que estos no aportaran al sistema dadbo la poca cantidad de datos legibles

In [174]:
#Elminación de los datos que no se tendrán en cuenta:
clasificacion=['Estado','Edad','Ubicación del caso','Nombre del país','Fecha de recuperación','Fecha de muerte','fecha reporte web','ID de caso','Fecha de notificación','Nombre departamento','Nombre municipio','Nombre del país','Tipo de recuperación','Nombre del grupo étnico','Código DIVIPOLA departamento','Unidad de medida de edad','Código ISO del país']
de=datos.drop(clasificacion, axis=1)
de.isna().sum()

Código DIVIPOLA municipio         0
Edad                              0
Sexo                              0
Tipo de contagio                  0
Recuperado                     3774
Fecha de inicio de síntomas    1486
Fecha de diagnóstico            334
Pertenencia étnica               30
Edad en días                      0
Edad en meses                     0
Edad en años                      0
dtype: int64

In [175]:
de = de.dropna() #Elimina las filas con NAN en alguna de sus columnas, de esta manera limpiamos los datos que no se usarán.
de = de.reset_index(drop=True) #Hace un reset al index del dataFrame para que este se pueda recorrer
de.isna().sum()

Código DIVIPOLA municipio      0
Edad                           0
Sexo                           0
Tipo de contagio               0
Recuperado                     0
Fecha de inicio de síntomas    0
Fecha de diagnóstico           0
Pertenencia étnica             0
Edad en días                   0
Edad en meses                  0
Edad en años                   0
dtype: int64

In [176]:
de['Tiempo de diagnostico'] = 1 #Crear la nueva columna para el valor desde el inicio de los sintomas hasta el diagnostico

In [177]:
#Usado para operar los valores de las fechas
from datetime import datetime
#For para operar el tiempo que transcurre desde que los sintomas aparecen hasta que se le diagnostica al paciente la enfermedad
for i in range(de['Tiempo de diagnostico'].value_counts().sum()):
    de['Tiempo de diagnostico'][i] = datetime.strptime(de['Fecha de diagnóstico'][i], "%Y-%m-%d %H:%M:%S").date() - datetime.strptime(de['Fecha de inicio de síntomas'][i], "%Y-%m-%d %H:%M:%S").date()
    

C:\Users\fredy\AppData\Local\Temp\ipykernel_19160\815131186.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de['Tiempo de diagnostico'][i] = datetime.strptime(de['Fecha de diagnóstico'][i], "%Y-%m-%d %H:%M:%S").date() - datetime.strptime(de['Fecha de inicio de síntomas'][i], "%Y-%m-%d %H:%M:%S").date()


In [178]:
fechas = ['Fecha de inicio de síntomas','Fecha de diagnóstico']
de = de.drop(fechas, axis=1)
de = de.reset_index(drop=True)
de

,Código DIVIPOLA municipio,Edad,Sexo,Tipo de contagio,Recuperado,Pertenencia étnica,Edad en días,Edad en meses,Edad en años,Tiempo de diagnostico
0,"5,001",34,F,Relacionado,Recuperado,6.0,0.0,0.0,34.0,"4 days, 0:00:00"
1,"5,001",20,F,Relacionado,Recuperado,6.0,0.0,0.0,20.0,"4 days, 0:00:00"
2,"5,360",46,F,Relacionado,Recuperado,6.0,0.0,0.0,46.0,"2 days, 0:00:00"
3,"5,088",40,M,Comunitaria,Recuperado,6.0,0.0,0.0,40.0,"2 days, 0:00:00"
4,"5,360",38,M,Relacionado,Recuperado,6.0,0.0,0.0,38.0,"4 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...
945490,"5,001",47,M,Comunitaria,Recuperado,6.0,0.0,0.0,47.0,"6 days, 0:00:00"
945491,"5,001",35,M,Comunitaria,Recuperado,6.0,0.0,0.0,35.0,0:00:00
945492,"5,360",65,F,Comunitaria,Recuperado,6.0,0.0,0.0,65.0,0:00:00
945493,"5,001",39,F,Comunitaria,Recuperado,6.0,0.0,0.0,39.0,"2 days, 0:00:00"


In [180]:
de.head()

,Código DIVIPOLA municipio,Edad,Sexo,Tipo de contagio,Recuperado,Pertenencia étnica,Edad en días,Edad en meses,Edad en años,Tiempo de diagnostico
0,"5,001",34,F,Relacionado,Recuperado,6.0,0.0,0.0,34.0,"4 days, 0:00:00"
1,"5,001",20,F,Relacionado,Recuperado,6.0,0.0,0.0,20.0,"4 days, 0:00:00"
2,"5,360",46,F,Relacionado,Recuperado,6.0,0.0,0.0,46.0,"2 days, 0:00:00"
3,"5,088",40,M,Comunitaria,Recuperado,6.0,0.0,0.0,40.0,"2 days, 0:00:00"
4,"5,360",38,M,Relacionado,Recuperado,6.0,0.0,0.0,38.0,"4 days, 0:00:00"


<h3>Transformación de datos.

In [181]:
##Ahora para el punto 5, tenemos que tranformar la informacion que tenemos en informacion que pueda entender la maquina, por ejemplo el sexo:
de['Sexo'].replace({'M':1 , 'F':0} , inplace=True) 
##Las fechas las usamos en formato str
de['Tiempo de diagnostico'] = de['Tiempo de diagnostico'].astype('str')

In [182]:
##Dependiendo del tipo de contagio se clasifican de 0 a 2
de['Tipo de contagio'].replace({'Relacionado': 0 , 'Comunitaria': 1, 'Importado': 2} , inplace=True) 

In [183]:
# El dataset contenía dualidad entre `fallecido` y `Fallecido`, por tanto se deja solo un parámetro.
# En recuperado lo que se tiene es el la actividad del paciente, o sea, si sigue o no contagiado, por lo que se debe tener
# en cuenta, no solo los fallecidos o recuperados, sino tambien los que aun siguen con el virus activo.
de['Recuperado'].replace({'fallecido':'Fallecido'},inplace=True)
de['Recuperado'].replace({'Fallecido': 0, 'Recuperado': 1, 'Activo': 2},inplace=True)

In [14]:
#Esra linea ya no se usaría entonces, por lo que los datos nulos ya fueron eliminados antes.
de['Recuperado'].fillna(method='ffill',inplace=True)
de['Pertenencia étnica'].fillna(method='ffill',inplace=True)

In [184]:
de

,Código DIVIPOLA municipio,Edad,Sexo,Tipo de contagio,Recuperado,Pertenencia étnica,Edad en días,Edad en meses,Edad en años,Tiempo de diagnostico
0,"5,001",34,0,0,1,6.0,0.0,0.0,34.0,"4 days, 0:00:00"
1,"5,001",20,0,0,1,6.0,0.0,0.0,20.0,"4 days, 0:00:00"
2,"5,360",46,0,0,1,6.0,0.0,0.0,46.0,"2 days, 0:00:00"
3,"5,088",40,1,1,1,6.0,0.0,0.0,40.0,"2 days, 0:00:00"
4,"5,360",38,1,0,1,6.0,0.0,0.0,38.0,"4 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...
945490,"5,001",47,1,1,1,6.0,0.0,0.0,47.0,"6 days, 0:00:00"
945491,"5,001",35,1,1,1,6.0,0.0,0.0,35.0,0:00:00
945492,"5,360",65,0,1,1,6.0,0.0,0.0,65.0,0:00:00
945493,"5,001",39,0,1,1,6.0,0.0,0.0,39.0,"2 days, 0:00:00"
